# PyMongo
The first step when working with PyMongo is to create a MongoClient to the running mongod instance.

Make sure you have a MongoDB instance running - see [https://www.mongodb.com/docs/manual/administration/install-community/](https://www.mongodb.com/docs/manual/administration/install-community/)

In [1]:
try:
    from pymongo import MongoClient
except:
    !pip install pymongo
    from pymongo import MongoClient

try:
    import psutil
except:
    !pip install psutil
    import psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.4/786.4 kB 9.6 MB/s eta 0:00:00


Being a local server, you can create a client in several ways.

In [2]:
client = MongoClient()
# same as 
#  client = MongoClient('localhost', 27017)
# or 
#  client = MongoClient('mongodb://localhost:27017/')

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

## Databases 
A single client instance of MongoDB can support multiple independent databases. When working with PyMongo you access databases using **attribute style access** on MongoClient instances.

So, the next line will "connect" to (or create if it does not exist) `sensorsDB` database.

This also means that you have to be very careful with the naming.

In [3]:
db = client.sensorsDB

## Collections 
A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database.

In [4]:
sensors_location = db.sensors_locations

An important note about collections (and databases) in MongoDB is that they are created lazily - none of the above commands have actually performed any operations on the MongoDB server. Collections and databases are created when the first document is inserted into them.

# Insert documents

To **insert a document** into a collection we can use the `insert_one()` method.

One way is to create a document and pass it to the insert_one() method.

In [5]:
data = {
        'location_name': 'Prometheus Server', 
        'description' : 'Prometheus Server @ lab. 163 / ISE /UAlg',
        'sensor': [ 
                    {
                        'sensor_name' : 'cpu_sensor', 
                        'unit' : 'percent'
                    },
                    {
                        'sensor_name' : 'mem_sensor', 
                        'unit' : 'percent'
                    }
             ]
       }

The `insert_one()` method takes a document as its argument and returns an instance of the inserted document.

In [6]:
x = sensors_location.insert_one(data)
x

InsertOneResult(ObjectId('67b482533e54caa329289d9d'), acknowledged=True)

Further, we can get the `_id` of the inserted document. This is relevant when we want to use it later to update or delete the document or to relate it to other documents.

The `_id` is a unique identifier for the document and is generated by the MongoDB server. The value is a 12-byte `ObjectId` which is generated based on the following components:

- Timestamp: The first 4 bytes are a timestamp, representing the ObjectId's creation, measured in seconds since the Unix epoch. This provides the `ObjectId` with a natural order by time of creation.

- Machine Identifier: The next 3 bytes are a unique identifier for the machine or process that generated the `ObjectId`. In older versions of MongoDB, this was the machine's hostname; in newer versions, it's a random value generated once per process. This ensures that ObjectIds generated on different machines or processes are likely to be unique.

- Process ID: The next 2 bytes are the process ID that generated the `ObjectId`. This further disambiguates ObjectIds created simultaneously on the same machine or by processes with the same machine identifier.

- Counter: The last 3 bytes are a counter, starting with a random value. This counter increments with each new ObjectId generated. It helps ensure uniqueness for ObjectIds created in the same second, on the same machine, by the same process.

The combination of these components ensures that each ObjectId is unique across different machines, processes, and moments in time. This system avoids the need for a more costly centralized ID generation scheme and makes it easy to generate IDs in a distributed environment, which is crucial for the scalability of MongoDB.

The 12-byte ObjectId format is compact and efficient, both in terms of storage space and performance. It also provides some level of timestamp-based sorting, which can be useful in certain applications.

In [7]:
location_id = x.inserted_id
location_id

ObjectId('67b482533e54caa329289d9d')

Let us see what is on the `sensors_location` collection. To list all documents in the collection we can use the `find()` method which returns a cursor that can be used to iterate over the documents.

In [10]:
from pprint import *

for doc in sensors_location.find():
    pprint(doc)

{'_id': ObjectId('65e63f25408da99fc5b59945'),
 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
 'location_name': 'Prometheus Server',
 'sensor': [{'sensor_name': 'cpu_sensor', 'unit': 'percent'},
            {'sensor_name': 'mem_sensor', 'unit': 'percent'}]}
{'_id': ObjectId('65e641207c453fc2910882c7'),
 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
 'location_name': 'Prometheus Server',
 'sensor': [{'sensor_name': 'cpu_sensor', 'unit': 'percent'},
            {'sensor_name': 'mem_sensor', 'unit': 'percent'}]}
{'_id': ObjectId('67b482533e54caa329289d9d'),
 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
 'location_name': 'Prometheus Server',
 'sensor': [{'sensor_name': 'cpu_sensor', 'unit': 'percent'},
            {'sensor_name': 'mem_sensor', 'unit': 'percent'}]}


And now, on the `sensors_readings`collection, we can insert on document for each reading of the sensor.

Note: we'll use the `datetime` module to generate the timestamp and the `psutil` module to generate the value. The latest, psutil, is a package that provides access to many different system utilities, e.g., CPU usage, memory usage, disk usage, network usage etc. 

In [11]:
import datetime
import psutil

for _ in range(200):
    # creat the document
    data = {
           'sensor' : {'location_id': location_id, 
                       'sensor_name' : 'cpu_sensor' 
                      },
            'value' : psutil.cpu_percent(interval=0.1),
            'units' : 'percent',
            'timestamp' : datetime.datetime.utcnow()
           }
    # send the document to the database
    res = db.sensors_readings.insert_one(data)
    print('.', end='')   

........................................................................................................................................................................................................

let us store the last `_id` for latter

In [12]:
_id = res.inserted_id
_id

ObjectId('67b482fb3e54caa329289e65')

To list all inserted readings, we can use again the `find()` method and force the cursor to return all documents by calling the `list()` function

In [13]:
list(db.sensors_readings.find())

[{'_id': ObjectId('65e63f25408da99fc5b59946'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 41.5,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 460000)},
 {'_id': ObjectId('65e63f25408da99fc5b59947'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 39.2,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 596000)},
 {'_id': ObjectId('65e63f25408da99fc5b59948'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 0.0,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 703000)},
 {'_id': ObjectId('65e63f25408da99fc5b59949'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 30.6,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 810

We can also list the inserted readings, sorted by value and timestamp. The `sort()` method allows us to sort the results by one or more fields, in this case `value` and `timestamp`. It receives a list of tuples with the field name and the sort order.

In [14]:
list(
    db.sensors_readings.find().sort([
        ('value',-1),
        ('timestamp', -1)]
    )
)

[{'_id': ObjectId('65e63f30408da99fc5b599af'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 100.0,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 52, 743000)},
 {'_id': ObjectId('65e63f30408da99fc5b599ae'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 100.0,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 52, 637000)},
 {'_id': ObjectId('65e63f31408da99fc5b599b7'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 98.3,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 53, 594000)},
 {'_id': ObjectId('65e63f31408da99fc5b599b8'),
  'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
   'sensor_name': 'cpu_sensor'},
  'value': 82.9,
  'units': 'percent',
  'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 53, 

Given the `ObjectId` (we stored ir earlier), it is possible to get one specific document

In [15]:
pprint(list(db.sensors_readings.find({'_id': _id})))

[{'_id': ObjectId('67b482fb3e54caa329289e65'),
  'sensor': {'location_id': ObjectId('67b482533e54caa329289d9d'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2025, 2, 18, 12, 54, 19, 65000),
  'units': 'percent',
  'value': 19.1}]


Or find all documents with a value greater than 50%

In [16]:
query = {'value': {'$gt': 50}}
pprint(list(db.sensors_readings.find(query)))

[{'_id': ObjectId('65e641207c453fc2910882ca'),
  'sensor': {'location_id': ObjectId('65e641207c453fc2910882c7'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2024, 3, 4, 21, 46, 8, 926000),
  'units': 'percent',
  'value': 52.6},
 {'_id': ObjectId('65e641217c453fc2910882d1'),
  'sensor': {'location_id': ObjectId('65e641207c453fc2910882c7'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2024, 3, 4, 21, 46, 9, 662000),
  'units': 'percent',
  'value': 53.1},
 {'_id': ObjectId('65e6412e7c453fc29108834a'),
  'sensor': {'location_id': ObjectId('65e641207c453fc2910882c7'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2024, 3, 4, 21, 46, 22, 380000),
  'units': 'percent',
  'value': 53.7},
 {'_id': ObjectId('65e641207c453fc2910882c8'),
  'sensor': {'location_id': ObjectId('65e641207c453fc2910882c7'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2024, 3, 4, 21, 46, 8, 7140

In the last 5 minutes...

In [17]:
query = {'timestamp': {'$gt': datetime.datetime.utcnow() - datetime.timedelta(minutes=5)}}

pprint(list(db.sensors_readings.find(query)))

[{'_id': ObjectId('67b482e63e54caa329289d9e'),
  'sensor': {'location_id': ObjectId('67b482533e54caa329289d9d'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2025, 2, 18, 12, 53, 58, 16000),
  'units': 'percent',
  'value': 23.2},
 {'_id': ObjectId('67b482e63e54caa329289d9f'),
  'sensor': {'location_id': ObjectId('67b482533e54caa329289d9d'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2025, 2, 18, 12, 53, 58, 120000),
  'units': 'percent',
  'value': 14.5},
 {'_id': ObjectId('67b482e63e54caa329289da0'),
  'sensor': {'location_id': ObjectId('67b482533e54caa329289d9d'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2025, 2, 18, 12, 53, 58, 225000),
  'units': 'percent',
  'value': 22.2},
 {'_id': ObjectId('67b482e63e54caa329289da1'),
  'sensor': {'location_id': ObjectId('67b482533e54caa329289d9d'),
             'sensor_name': 'cpu_sensor'},
  'timestamp': datetime.datetime(2025, 2, 18, 12, 53, 58

## Embending of information I
In this approach, a single document contains **multiple sensors with a single read**. Also, embedded location info.

In [18]:
import datetime
import psutil

for _ in range(200):
    data = {
        'location_name': 'Prometheus Server', 
        'description' : 'Prometheus Server @ lab. 163 / ISE /UAlg',
        'sensors' : [ 
               {
                   'sensor_name' : 'mem_sensor', 
                   'value' : psutil.virtual_memory().percent,
                   'units' : 'percent'
               },
               {
                   'sensor_name' : 'cpu_sensor', 
                   'value' : psutil.cpu_percent(interval=0.1),
                   'units' : 'percent'
               }
           ],
        'timestamp' : datetime.datetime.utcnow()
    }
    db.sensors_readings.insert_one(data)
    print('.', end='')

........................................................................................................................................................................................................

Get the last insert. In this case we use the `limit(1)` method to limit the number of documents returned. Since the list is sorted by timestamp, the first one will be the last inserted

In [19]:
pprint(
    list(
        db.sensors_readings.find()\
            .sort([('timestamp', -1)])\
            .limit(1)
    )
)

[{'_id': ObjectId('67b483a93e54caa329289f2d'),
  'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
  'location_name': 'Prometheus Server',
  'sensors': [{'sensor_name': 'mem_sensor', 'units': 'percent', 'value': 73.3},
              {'sensor_name': 'cpu_sensor', 'units': 'percent', 'value': 18.3}],
  'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 13, 325000)}]


## Embending of information II
A single document contains multiple sensors - and multiple reads.

In [21]:
data = {
    'location_name': 'Prometheus Server', 
    'description' : 'Prometheus Server @ lab. 163 / ISE /UAlg',
    'sensors' : [ 
           {
               'sensor_name' : 'mem_sensor', 
               'values' :[] ,
               'units' : 'percent'
           },
           {
               'sensor_name' : 'cpu_sensor', 
               'values' : [],
               'units' : 'percent'
           }
       ],
}

# get the readingd id to latter add values to the readings
readings_id = db.sensors_readings.insert_one(data).inserted_id
readings_id

ObjectId('67b483c33e54caa329289f2f')

However, in this implementation the **full document is upload each time a new read is made**. Which means that we need to update the document in the database each time a new read is made. To do this, we use the `update_one()` method which takes 2 arguments: the query and the update. A third argument are options which you can find in the documentation ([https://www.mongodb.com/docs/manual/reference/method/db.collection.update/](https://www.mongodb.com/docs/manual/reference/method/db.collection.update/)).

In [22]:
for _ in range(100):
    mem = psutil.virtual_memory().percent
    cpu = psutil.cpu_percent(interval=0.01)

    # update the data
    data['sensors'][0]['values'].append({'value': mem, 'timestamp' : datetime.datetime.utcnow()})
    data['sensors'][1]['values'].append({'value': cpu, 'timestamp' : datetime.datetime.utcnow()})
    # update the database, sending the full document again!!
    db.sensors_readings.update_one(
        {'_id': readings_id}, 
        {'$set': data}
    )
    
    print('.', end='')

....................................................................................................

The last reading is 

In [23]:
x = list(
    db.sensors_readings.find() \
        .sort([('_id', -1)]) \
        .limit(1)
)
x

[{'_id': ObjectId('67b483c33e54caa329289f2f'),
  'location_name': 'Prometheus Server',
  'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
  'sensors': [{'sensor_name': 'mem_sensor',
    'values': [{'value': 73.2,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 406000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 420000)},
     {'value': 73.6,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 433000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 446000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 459000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 471000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 484000)},
     {'value': 73.5,
      'timestamp': datetime.datetime(2025, 2, 18, 12, 57, 48, 498000)},
     {'value': 73.5,
      'timestamp': datetime.datet

to get a value from it we can "navigate" the array/dictionary

In [24]:
x[0]['sensors'][0]['values'][0]['value']

73.2

## Embending of information III
As previously, a single document contains multiple sensors - and multiple reads. But now, only the fields we need are updated. 

In [25]:
import datetime
import psutil

data = {
    'location_name': 'Prometheus Server', 
    'description' : 'Prometheus Server @ lab. 163 / ISE /UAlg', 
    'sensors' : [ 
           {
               'sensor_name' : 'mem_sensor', 
               'values' : [],
               'units' : 'percent'
           },
           {
               'sensor_name' : 'cpu_sensor', 
               'values' : [],
               'units' : 'percent'
           }
       ]
}

readings_id = db.sensors_readings.insert_one(data).inserted_id

Now, a first document was inserted with no sensors values. The document `_id` was stored and in the following data will be appended/pushed to the corresponding document

In [26]:
for _ in range(200):
    mem = psutil.virtual_memory().percent
    cpu = psutil.cpu_percent(interval=0.1)
    
    # update the database, sending only the update
    db.sensors_readings.update_one(
        {'_id': readings_id}, 
        {
            '$push': {
                'sensors.0.values': {'value': mem, 'timestamp' : datetime.datetime.utcnow()},
                'sensors.1.values': {'value': cpu, 'timestamp' : datetime.datetime.utcnow()}        
            }
        }
    )    
    
    print('.', end='')

........................................................................................................................................................................................................

The last reading is 

In [27]:
pprint(
    list(
        db.sensors_readings\
            .find()\
            .sort([('_id', -1)])\
            .limit(1)
    )
)

[{'_id': ObjectId('67b483ef3e54caa329289f30'),
  'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
  'location_name': 'Prometheus Server',
  'sensors': [{'sensor_name': 'mem_sensor',
               'units': 'percent',
               'values': [{'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 32, 839000),
                           'value': 73.9},
                          {'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 32, 949000),
                           'value': 74.0},
                          {'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 33, 56000),
                           'value': 74.0},
                          {'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 33, 159000),
                           'value': 74.0},
                          {'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 33, 275000),
                           'value': 73.9},
                          {'timestamp': datetime.datetime(2025, 2, 18, 12, 58, 33, 377000),
       

# Getting Documents
Getting a single document with find_one() can be done using the `find_one()` method which returns the first document in the collection which matches the query. The syntax is the same as the `find()` method, i.e., `find_one({query}, {projection})` 

In [29]:
db.sensors_readings.find_one()

{'_id': ObjectId('65e63f25408da99fc5b59946'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
  'sensor_name': 'cpu_sensor'},
 'value': 41.5,
 'units': 'percent',
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 460000)}

Find one reading from "Prometheus Server"

In [30]:
db.sensors_readings.find_one({'location_name':'Prometheus Server'})

{'_id': ObjectId('65e63f3a408da99fc5b59a0e'),
 'location_name': 'Prometheus Server',
 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
 'sensors': [{'sensor_name': 'mem_sensor', 'value': 75.5, 'units': 'percent'},
  {'sensor_name': 'cpu_sensor', 'value': 0.0, 'units': 'percent'}],
 'timestamp': datetime.datetime(2024, 3, 4, 21, 38, 2, 916000)}

Get the Object id for one reading on the sensor's reading collection

In [31]:
obj_id = db.sensors_readings.find_one({'location_name':'Prometheus Server'})["_id"]
obj_id

ObjectId('65e63f3a408da99fc5b59a0e')

Querying By ObjectId

In [32]:
from bson.objectid import ObjectId

db.sensors_readings.find_one({'_id': obj_id})  # update the _id

{'_id': ObjectId('65e63f3a408da99fc5b59a0e'),
 'location_name': 'Prometheus Server',
 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg',
 'sensors': [{'sensor_name': 'mem_sensor', 'value': 75.5, 'units': 'percent'},
  {'sensor_name': 'cpu_sensor', 'value': 0.0, 'units': 'percent'}],
 'timestamp': datetime.datetime(2024, 3, 4, 21, 38, 2, 916000)}

Do projections, i.e., select which fields to present

In [33]:
db.sensors_readings.find_one(
    {'_id': obj_id},
    {'sensors':1}
)

{'_id': ObjectId('65e63f3a408da99fc5b59a0e'),
 'sensors': [{'sensor_name': 'mem_sensor', 'value': 75.5, 'units': 'percent'},
  {'sensor_name': 'cpu_sensor', 'value': 0.0, 'units': 'percent'}]}

# Bulk Insert
In addition to inserting a single document, we can also perform bulk insert operations, by passing a list as the first argument to insert_many(). This will insert each document in the list, sending only a single command to the server.

The result from insert_many() now returns multiple ObjectId instances, one for each inserted document.

In [34]:
new_posts = [{
                'sensor': {'location_id': ObjectId('5a95821bdc936e0cfc7c7d96'),
                'sensor_name': 'cpu_sensor'},
                'timestamp': datetime.datetime.utcnow(),
                'units': 'percent',
                'value': 4.5
            },
             {
                'sensor': {'location_id': ObjectId('5a95821bdc936e0cfc7c7d96'),
                'sensor_name': 'cpu_sensor'},
                'timestamp': datetime.datetime.utcnow(),
                'units': 'percent',
                'value': 4.5
             }
            ]
result = db.sensors_readings.insert_many(new_posts)
result

InsertManyResult([ObjectId('67b4845c3e54caa329289f31'), ObjectId('67b4845c3e54caa329289f32')], acknowledged=True)

and get the id's of the inserted objects

In [35]:
result.inserted_ids

[ObjectId('67b4845c3e54caa329289f31'), ObjectId('67b4845c3e54caa329289f32')]

# Querying for More Than One Document

To get more than a single document as the result of a query we use the find() method. find() returns a Cursor instance, which allows us to iterate over all matching documents. 

Note that do to our experiments, the documents no not follow any type of schema. This is not recommended but is possible. So, fields like timestamp are found in every document, but in different "positions" in the document. 

For example, we can iterate over every document in the posts collection:

In [36]:
for doc in db.sensors_readings.find():
    pprint(doc)

{'_id': ObjectId('65e63f25408da99fc5b59946'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 460000),
 'units': 'percent',
 'value': 41.5}
{'_id': ObjectId('65e63f25408da99fc5b59947'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 596000),
 'units': 'percent',
 'value': 39.2}
{'_id': ObjectId('65e63f25408da99fc5b59948'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 703000),
 'units': 'percent',
 'value': 0.0}
{'_id': ObjectId('65e63f25408da99fc5b59949'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 810000),
 'units': 'percent

you can also limit the output and order it...

In [37]:
for doc in db.sensors_readings.find().sort([('_id',1)]).limit(2):
    pprint(doc)

{'_id': ObjectId('65e63f25408da99fc5b59946'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 460000),
 'units': 'percent',
 'value': 41.5}
{'_id': ObjectId('65e63f25408da99fc5b59947'),
 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'),
            'sensor_name': 'cpu_sensor'},
 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 596000),
 'units': 'percent',
 'value': 39.2}


## Counting
If we just want to know how many documents match a query we can perform a count_documents() operation instead of a full query.

In [38]:
db.sensors_readings.count_documents({})

1213

It is also possible to count the number of documents in a collection satisfying a query

In [39]:
query = {'location_name':'Prometheus Server'}
db.sensors_readings.count_documents(query)

607

## Range Queries
MongoDB supports many different types of advanced queries.


In [40]:
last_inserted_doc_timestamp = db.sensors_readings.find().sort([('_id',-1)]).limit(1)[0]['timestamp']
last_inserted_doc_timestamp

datetime.datetime(2025, 2, 18, 13, 0, 12, 279000)

In [41]:
for doc in db.sensors_readings.find({'timestamp': last_inserted_doc_timestamp}):
    print(doc)

{'_id': ObjectId('67b4845c3e54caa329289f31'), 'sensor': {'location_id': ObjectId('5a95821bdc936e0cfc7c7d96'), 'sensor_name': 'cpu_sensor'}, 'timestamp': datetime.datetime(2025, 2, 18, 13, 0, 12, 279000), 'units': 'percent', 'value': 4.5}
{'_id': ObjectId('67b4845c3e54caa329289f32'), 'sensor': {'location_id': ObjectId('5a95821bdc936e0cfc7c7d96'), 'sensor_name': 'cpu_sensor'}, 'timestamp': datetime.datetime(2025, 2, 18, 13, 0, 12, 279000), 'units': 'percent', 'value': 4.5}


As an example, lets perform a query where we limit results to readings insertd in the last 5 minutes:

In [42]:
date = datetime.datetime.utcnow() - datetime.timedelta(minutes=5)
query = {'timestamp': {'$gt': date}}
for doc in db.sensors_readings.find(query):
    print(doc)

{'_id': ObjectId('67b483943e54caa329289e66'), 'location_name': 'Prometheus Server', 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg', 'sensors': [{'sensor_name': 'mem_sensor', 'value': 73.2, 'units': 'percent'}, {'sensor_name': 'cpu_sensor', 'value': 18.3, 'units': 'percent'}], 'timestamp': datetime.datetime(2025, 2, 18, 12, 56, 52, 254000)}
{'_id': ObjectId('67b483943e54caa329289e67'), 'location_name': 'Prometheus Server', 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg', 'sensors': [{'sensor_name': 'mem_sensor', 'value': 73.5, 'units': 'percent'}, {'sensor_name': 'cpu_sensor', 'value': 0.0, 'units': 'percent'}], 'timestamp': datetime.datetime(2025, 2, 18, 12, 56, 52, 356000)}
{'_id': ObjectId('67b483943e54caa329289e68'), 'location_name': 'Prometheus Server', 'description': 'Prometheus Server @ lab. 163 / ISE /UAlg', 'sensors': [{'sensor_name': 'mem_sensor', 'value': 73.5, 'units': 'percent'}, {'sensor_name': 'cpu_sensor', 'value': 0.0, 'units': 'percent'}], 'timesta

All readings witha `value` lower than 10

In [43]:
for doc in db.sensors_readings.find({'value': {'$lt': 10}}):
    print(doc)

{'_id': ObjectId('65e63f25408da99fc5b59948'), 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'), 'sensor_name': 'cpu_sensor'}, 'value': 0.0, 'units': 'percent', 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 41, 703000)}
{'_id': ObjectId('65e63f26408da99fc5b5994d'), 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'), 'sensor_name': 'cpu_sensor'}, 'value': 0.0, 'units': 'percent', 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 42, 236000)}
{'_id': ObjectId('65e63f26408da99fc5b5994e'), 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'), 'sensor_name': 'cpu_sensor'}, 'value': 0.0, 'units': 'percent', 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 42, 345000)}
{'_id': ObjectId('65e63f26408da99fc5b5994f'), 'sensor': {'location_id': ObjectId('65e63f25408da99fc5b59945'), 'sensor_name': 'cpu_sensor'}, 'value': 0.0, 'units': 'percent', 'timestamp': datetime.datetime(2024, 3, 4, 21, 37, 42, 449000)}
{'_id': ObjectId('65e63f26408da99fc5b59950'), 's

All CPU readings with `value` lower than 10 in the next type of documents.
[be aware that, since we were experimenting, the documents may not have the same structure]

![./images/doc_example.png](./images/doc_example.png)

In [ ]:
for doc in db.sensors_readings.find({'sensors.1.values.value': {'$gt': 10}}):
    print(doc)